# NOAA Analysis

In [106]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [47]:
event_details = pd.read_csv("results/combined_0.csv", low_memory=False);
event_details.shape

(1457423, 51)

In [7]:
event_details.head(3)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.2,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.6,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.58,-75.7,40.65,-75.47,NaN,NaN,PUB


### Problem 1 - summarize the property damage, by state, by year

In [57]:
event_details_state['DAMAGE_PROPERTY'].head(3)

0    250K
1     25K
2     25K
Name: DAMAGE_PROPERTY, dtype: object

The property damage column is not in number form. A quick glance shows that this field is suffixed with K representing thousand and M representing million. Next, we will convert this into number form.

In [64]:
# stolen from https://stackoverflow.com/questions/39684548/convert-the-string-2-90k-to-2900-or-5-2m-to-5200000-in-pandas-dataframe
def value_to_float(x):
    x = str(x)
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

250000.0

In [71]:
event_details['DAMAGE_PROPERTY_DOLLARS'] = event_details['DAMAGE_PROPERTY'].apply(value_to_float)

In [72]:
event_details['STATE'].unique()

array(['OKLAHOMA', 'TEXAS', 'PENNSYLVANIA', 'NEBRASKA', 'MISSISSIPPI',
       'NEW MEXICO', 'ARKANSAS', 'MISSOURI', 'CONNECTICUT', 'FLORIDA',
       'NORTH CAROLINA', 'ALABAMA', 'KENTUCKY', 'MARYLAND', 'MINNESOTA',
       'SOUTH DAKOTA', 'IOWA', 'LOUISIANA', 'OHIO', 'KANSAS',
       'NORTH DAKOTA', 'INDIANA', 'COLORADO', 'SOUTH CAROLINA',
       'WEST VIRGINIA', 'WYOMING', 'GEORGIA', 'WISCONSIN', 'ILLINOIS',
       'TENNESSEE', 'NEW JERSEY', 'MICHIGAN', 'CALIFORNIA',
       'MASSACHUSETTS', 'NEW HAMPSHIRE', 'OREGON', 'VIRGINIA', 'ARIZONA',
       'NEW YORK', 'MONTANA', 'MAINE', 'VERMONT', 'UTAH', 'DELAWARE',
       'IDAHO', 'WASHINGTON', 'HAWAII', 'RHODE ISLAND', 'NEVADA',
       'PUERTO RICO', 'DISTRICT OF COLUMBIA', 'ALASKA', 'VIRGIN ISLANDS',
       'GUAM', 'AMERICAN SAMOA', 'ATLANTIC SOUTH', 'LAKE ERIE',
       'LAKE HURON', 'GULF OF MEXICO', 'LAKE ST CLAIR', 'E PACIFIC',
       'HAWAII WATERS', 'LAKE MICHIGAN', 'ATLANTIC NORTH', 'LAKE ONTARIO',
       'ST LAWRENCE R', 'LAKE SUPERI

We see that there are non-states in our dataset, we need to be sure to filter these out.

In [74]:
states = pd.read_csv("states.csv")
#upper all states so they match file
states['State'] = states['State'].apply(lambda x: x.upper())
event_details_state = event_details[event_details['STATE'].isin(states['State'])]
event_details_state.shape

(1424943, 52)

We went from 1,457,423 to 1,424,943 after filtering non US states.

In [100]:
event_details_grouped = event_details_state.groupby(['STATE','YEAR'])['DAMAGE_PROPERTY_DOLLARS'].sum()

In [105]:
event_details_grouped.head()

STATE    YEAR
ALABAMA  1950     27500.000
         1951     35000.000
         1952   5452500.000
         1953   3070000.000
         1954    607530.000
Name: DAMAGE_PROPERTY_DOLLARS, dtype: float64

In [104]:
# outputs to csv file
event_details_grouped.to_csv("property_damage_summary.csv")